In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

In [3]:
df.isnull().any()

In [4]:
df.describe()

In [5]:
df.dtypes

In [6]:
df.drop('customerID', axis=1, inplace=True)

In [7]:
df.dtypes

In [8]:
df.loc[df['TotalCharges'] == ' ']

In [9]:
df.loc[(df['TotalCharges'] == ' '), 'TotalCharges'] = 0

In [10]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
df.dtypes

In [11]:
df['Churn'].replace(to_replace ='Yes', value=1, inplace=True)
df['Churn'].replace(to_replace = 'No', value=0, inplace=True)
df = pd.get_dummies(df)

In [12]:
df.head()

In [13]:
df.describe()

In [14]:
X = df.drop('Churn', axis = 1)
y = df['Churn']

In [15]:
X

In [16]:
np.unique(y)

In [17]:
np.sum(y)

In [18]:
np.sum(1-y)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [21]:
X_train.shape

In [22]:
X_test.shape

In [23]:
np.sum(y_train)

In [24]:
np.sum(1 - y_train)

In [25]:
np.sum(y_test)

In [26]:
np.sum(1-y_test)

In [35]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
X_res, y_res = sm.fit_resample(X_train, y_train)

clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))), 
  ('classification', ExtraTreesClassifier(random_state=0))
])

In [46]:
from sklearn.model_selection import GridSearchCV
params = {
    'classification__n_estimators' : [10, 20, 50, 100, 200],
}
best_clf = GridSearchCV(clf, params ,cv=10)
best_clf.fit(X_res, y_res)

In [47]:
best_clf.best_score_

In [48]:
best_clf.best_params_

In [49]:
y_predict = best_clf.predict(X_test)
sum(y_predict==y_test)/len(y_test)

In [50]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['Unchange', 'Change']
C = confusion_matrix(y_test,y_predict) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C, annot=True, fmt=".2f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()